In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm  # Import tqdm

def scrape_weather(year, month):
    url = f"https://www.timeanddate.com/weather/vietnam/ho-chi-minh/historic?month={month}&year={year}"

    # Gửi yêu cầu GET để tải trang
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # Tìm dropdown chứa các ngày
    select_element = soup.find("select", {"id": "wt-his-select"})
    options = select_element.find_all("option")

    data = []
    
    # Lặp qua từng option (ngày) trong dropdown
    for option in options:
        option_date = option["value"]
        if option_date == "":  # Nếu không có giá trị, bỏ qua
            continue
        
        # Tạo URL có tham số ngày để lấy dữ liệu cho từng ngày
        url_for_day = f"https://www.timeanddate.com/weather/vietnam/ho-chi-minh/historic?month={month}&year={year}&hd={option_date}"
        
        # Lấy dữ liệu cho ngày cụ thể
        res_day = requests.get(url_for_day)
        soup_day = BeautifulSoup(res_day.text, "html.parser")
        
        # Trích xuất bảng dữ liệu thời tiết
        table = soup_day.find("table", {"id": "wt-his"})
        if table:
            rows = table.tbody.find_all("tr")
            for row in rows:
                cols = row.find_all("td")
                if len(cols) == 0:
                    continue

                time_of_day = row.find("th").text.strip()
                if time_of_day == "00:00":
                    continue
                temp = cols[1].text.replace("°C", "").strip()
                weather = cols[2].text.strip()
                wind = cols[3].text.strip()
                wind_dir_tag = cols[4].find("span")
                wind_direction = wind_dir_tag["title"] if wind_dir_tag and wind_dir_tag.has_attr("title") else ""
                humidity = cols[5].text.strip()
                barometer = cols[6].text.strip()
                visibility = cols[7].text.strip().replace("N/A", "")

                data.append({
                    "Date": pd.to_datetime(option_date, format='%Y%m%d').date(),
                    "Time": time_of_day,
                    "Temperature (°C)": temp,
                    "Weather": weather,
                    "Wind": wind,
                    "Wind Direction": wind_direction,
                    "Humidity": humidity,
                    "Barometer (mbar)": barometer,
                    "Visibility (km)": visibility,
                })
    
    return pd.DataFrame(data)


# Giai đoạn tải dữ liệu chia thành batch 2 năm
start_date = datetime(2015, 1, 1)
end_date = datetime(2025, 4, 1)
all_data = pd.DataFrame()

# Dùng tqdm để hiển thị tiến trình cho từng batch
while start_date <= end_date:
    batch_end_date = start_date.replace(year=start_date.year + 2)
    if batch_end_date > end_date:
        batch_end_date = end_date

    print(f"Scraping data for {start_date.strftime('%Y-%m')} to {batch_end_date.strftime('%Y-%m')}")
    
    # Thu thập dữ liệu cho khoảng thời gian hiện tại (2 năm)
    batch_data = pd.DataFrame()
    current_date = start_date
    
    # Dùng tqdm cho vòng lặp qua các tháng trong batch
    for current_date in tqdm(pd.date_range(start=start_date, end=batch_end_date, freq='MS'), desc="Months", unit="month"):
        monthly_df = scrape_weather(current_date.year, current_date.month)
        batch_data = pd.concat([batch_data, monthly_df], ignore_index=True)
    
    # Lưu dữ liệu của batch vào file CSV riêng
    batch_filename = f"weather_data_{start_date.year}_{batch_end_date.year}.csv"
    batch_data.to_csv(batch_filename, index=False, encoding="utf-8")
    print(f"Data saved to {batch_filename}")
    
    # Cập nhật start_date cho batch tiếp theo
    start_date = batch_end_date + timedelta(days=1)

print("All batches processed.")


Scraping data for 2015-01 to 2017-01


Months: 100%|██████████| 25/25 [03:46<00:00,  9.08s/month]


Data saved to weather_data_2015_2017.csv
Scraping data for 2017-01 to 2019-01


Months: 100%|██████████| 24/24 [03:14<00:00,  8.10s/month]


Data saved to weather_data_2017_2019.csv
Scraping data for 2019-01 to 2021-01


...

Data saved to weather_data_2021_2023.csv
Scraping data for 2023-01 to 2025-01


Months: 100%|██████████| 24/24 [03:39<00:00,  9.15s/month]


Data saved to weather_data_2023_2025.csv
Scraping data for 2025-01 to 2025-04


Months: 100%|██████████| 3/3 [00:20<00:00,  6.78s/month]

Data saved to weather_data_2025_2025.csv
All batches processed.
